
## 2) Web scraping vs APIs  - what's the difference?
When you access data on the web, you typically download a resource. This can occur on a browser, or in your Python console. Because our interaction is primarily visual, information returned is in HTML, a markup language, that delivers both content and rules about how the content is to be presented (fonts, text size, bold, arrangement). By contrast, APIs typically are built to only return data. For this reason, the data is typically returned in XML or JSON formats. The following are examples of each: [examples here] 

Similarly, saving a web page or going to Source in Developer Tools allows you to view the html code associated with each. 


### Note on Robots:
We also hear a lot about robots. A robot is ... to accomplish any kind of automated task. However, if you... Bots can be built to extract data from APIs or web scraping. Note that accessing APIs through the console does not necessarily mean it is a bot. If you manually send requests on the console to download specific resources, that is not a bot. Requires that it be automated. In the next section, we will discuss a text file called "robots.txt" that is typically contained in the root folder, that contains instructions to bots 

For instance, a bot can click through every post on a forum, downloading files for each looking for a specific word or text. In our example, we show how to do this for Wikipedia. 

## 3) Menagerie of tools: crawlers, spiders, scrapers - what's the difference? 
Web crawlers or spiders comes from. An image of long, spindly legs, traversing from hyperlink to hyperlink. It is these automated crawlers that continually traverse the web and index new or changed content, that search engines used to update and present the most relevant results to your search requests. 

Web scraping is a little different. While many of the tools used may be identical or similar, web scraping "focuses more on the transformation of unstructured data on the web, typically in HTML format, into structured data that can be stored and analyzed in a central local database or spreadsheet." (https://en.wikipedia.org/wiki/Web_scraping) 

In many cases, to the server, these processes look somewhat identical. Resources are sent in response to requests. Rather, it is what is done to those resources after they are sent, and the overall goal, that differentiates web crawling and scraping. Most websites want crawlers to find them so their pages appear on popular search engines, but see no clear-cut benefit when their content is parsed and converted into usable data. Beyond research, many companies also use web scraping (in a legal grey area or illegally) to repurpose content, etc, a real estate website scraping data from Craigslist to re-post as listings on their website.

## 4) Considerate robots and legality 
The use of robots.txt file is a convention. 

identifies the links, and highlights specific crawlers. Let's take a look at reddit's [insert reddit's privacy policy]

Twitter's privacy policy. [insert Twitter privacy policy]  

Blogs, for instance, or many forum sites do not have APIs available. 

Most frequent is getting your IP blocked temporarily or permanently. In addition, if you plan to publish your results for research, contacting the agency is probably a good idea. 

In summary:   
1) Find the websites' robots.txt and do not access those pages through your bot  
2) Make sure your bot does not make too many requests in a specific period (etc. by using Python's sleep.wait function)   
3) Look up the website's term of use or terms of service.   


## Step 1: Examining the webpage structure

The first step is to examine the webpage in your browser, using developer tools (Firefox/Chrome). First, identify the element that you want to pull data from. In this case, a series of links. Forum traversal. 

For this case, we concentrate on the box that appears at the side of each of the company's pages. 

While we've identified visually where we want to pull the element from, this may or may not translate into code. In our case, thankfully, the pages have similar enough structure. HTML has an optional category called "class", which, among other uses, allows the website to specifiy how the formatting of an element should look (using what is called css). For our purposes, we can use the "infobox vcard" class to tell the program which box we want to pull out and use.

[click inspect element] 




## Step 2: Interacting with the webpage through the console 

After examining the webpage structure through your browser, now it's time to interact with the underlying html code (what you see in the inspect element page) directly in your console. Both processes are useful to coming up with a strategy of how (and whether) data from the website can be scraped. 

First, import urllib2 and BeautifulSoup. Downloading a html copy of the site is as simple as: 